The Cogo dataset (JSON) has just been referred to as JSON; and the Liveworks dataset has just been referred to as SQL. You shall see variables like sql_dict, json_dict which you should understand to belong to the Cogo and Liveworks datasets respectively.

In [101]:
# SQL part. abstraction to get json data, in a windowed way
def get_next_sql(sql_iter=0, step_size=10000):
    import pymysql

    connection = pymysql.connect(host='data-engineer-rds.czmkgxqloose.us-east-1.rds.amazonaws.com',
                                 user='cogo_read_only',
                                 password='N&f#vSq9',
                                 db='liveworks',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

    table_name = 'cogo_list_v1'
    try:
        with connection.cursor() as cursor:
            cursor.execute("select * from {} limit {}, {} ".format(table_name, sql_iter, step_size))
            return cursor.fetchall()
    finally:
        connection.close()

In [102]:
get_next_sql(0, 5)

[{'address': '7294 Nicole Motorway Phamview, PR 35990-7992',
  'birthdate': '1998-03-08',
  'company': 'Hicks-Bennett',
  'emd5': '0000141445c93deb94819729f172b9e3',
  'id': 1,
  'job': 'Veterinary surgeon',
  'name': 'Eric Brooks',
  'sex': 'M'},
 {'address': '7491 Mitchell Centers Davisstad, MO 95858-8608',
  'birthdate': '1997-03-02',
  'company': 'Bender PLC',
  'emd5': '000019ae599d24237e28e67cf6655f55',
  'id': 2,
  'job': 'Hotel manager',
  'name': 'Mrs. Nicole Payne',
  'sex': 'F'},
 {'address': '001 Macias Lodge Suite 314 South Summerfurt, WI 84980',
  'birthdate': '1994-02-23',
  'company': 'Diaz Ltd',
  'emd5': '00001f81bfbc733f17fb6ee34fc0d24d',
  'id': 3,
  'job': 'Immigration officer',
  'name': 'Katherine Shaffer',
  'sex': 'F'},
 {'address': '4036 Cisneros Mount Leonhaven, NY 49681-7671',
  'birthdate': '2007-03-09',
  'company': 'Alvarado, Williams and Shah',
  'emd5': '0000634857164b38b640b3032bc7ac52',
  'id': 4,
  'job': 'Aeronautical engineer',
  'name': 'Marie Car

In [103]:
# JSON part
import json
def make_request_for_json_page(json_page):
    import requests
    return json.loads(requests.get("http://ec2co-ecsel-mt18uepnps93-1308701016.us-east-1.elb.amazonaws.com:8000/records?page={}".format(json_page)).text)

def get_next_json(json_page=1):
    return make_request_for_json_page(json_page)['rows']

In [104]:
get_next_json(1)

[{'address': '87826 Ford Canyon Reeseport, WI 70426',
  'birthdate': '1971-02-22',
  'company': 'May, Lee and Jackson',
  'emd5': '00001552528e3519238cb2929d47022e',
  'id': 1,
  'job': 'Advice worker',
  'name': 'Vanessa Thomas',
  'sex': 'F'},
 {'address': 'USNS Lester FPO AE 63607-7210',
  'birthdate': '2017-02-08',
  'company': 'Richard, Cole and Smith',
  'emd5': '00001f81bfbc733f17fb6ee34fc0d24d',
  'id': 2,
  'job': 'Patent attorney',
  'name': 'Christine Sutton',
  'sex': 'F'},
 {'address': '4974 Gonzalez Ports Valeriemouth, PW 52121-1897',
  'birthdate': '2000-12-10',
  'company': 'Hess, Andrade and Wright',
  'emd5': '00002d1b38a2da8fd6310032fd1e58de',
  'id': 3,
  'job': 'Architect',
  'name': 'Joseph Butler',
  'sex': 'M'},
 {'address': 'Unit 6272 Box 9634 DPO AA 12636',
  'birthdate': '2015-11-03',
  'company': 'Barber PLC',
  'emd5': '0000634857164b38b640b3032bc7ac52',
  'id': 4,
  'job': 'Translator',
  'name': 'Michael Griffin',
  'sex': 'M'},
 {'address': '6861 James R

In [143]:
def is_different(job_title1, job_title2):
    return job_title1.strip().lower() == job_title2.strip().lower()

def calculate_when_matching(json_dict, sql_dict, same_title, diff_title):
    if is_different(json_dict['job'], sql_dict['job']):
        diff_title += 1
    else:
        same_title += 1
    return same_title, diff_title

def populate_csv_list(json_dict, sql_dict, result):
    new_entry = {}
    new_entry['emd5'] = json_dict['emd5']
    
    new_entry['cogo.title'] = json_dict['job']
    new_entry['cogo.company'] = json_dict['company']
    
    new_entry['liveworks.title'] = sql_dict['job']
    new_entry['liveworks.company'] = sql_dict['company']
    
    result.append(new_entry)

def compare(json_dict, sql_dict):
    if json_dict['emd5'] == sql_dict['emd5']:
        return 0
    elif json_dict['emd5'] < sql_dict['emd5']:
        return -1
    elif json_dict['emd5'] > sql_dict['emd5']:
        return 1
    
def write_dict_to_csv(result, filename='result_cogo'):
    import csv
    filename_w_format = filename + '.csv'
    with open(filename_w_format, 'w') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quoting=csv.QUOTE_ALL)
        for row in result:
            writer.writerow(row.values())

In [141]:
STEP_SIZE = 10000 # only for the sql file. JSON is paginated anyway
json_window, sql_window = get_next_json(), get_next_sql()

json_iter, sql_iter = 0, 0
global_json_iter, global_sql_iter = 1, 0

left, right, both = 0, 0, 0 # count of data points only in json, only in sql and in both
same_title, diff_title = 0, 0

len_json_window, len_sql_window = None, None

result = []
while json_window and sql_window:
    len_json_window, len_sql_window = len(json_window), len(sql_window)
    
    if any(x is 0 for x in [len_json_window, len_sql_window]):
        break
        
    while json_iter < len_json_window and sql_iter < len_sql_window:
        if compare(json_window[json_iter], sql_window[sql_iter]) == 0:
            same_title, diff_title = calculate_when_matching(json_window[json_iter], sql_window[sql_iter], same_title, diff_title)
            populate_csv_list(json_window[json_iter], sql_window[sql_iter], result)
            
            both += 1
            json_iter += 1
            sql_iter += 1
        elif compare(json_window[json_iter], sql_window[sql_iter]) == -1:
            json_iter += 1
            left += 1
        elif compare(json_window[json_iter], sql_window[sql_iter]) == 1:
            sql_iter += 1
            right += 1
        
    if json_iter == len_json_window:
        global_json_iter += 1 # next page of json
        json_iter = 0
        json_window = get_next_json(global_json_iter)

    if sql_iter == len_sql_window:
        global_sql_iter += STEP_SIZE # next 10000 rows
        sql_iter = 0
        sql_window = get_next_sql(global_sql_iter)


if json_iter < len_json_window:
    json_iter -= 1
    while json_window:
        left += len_json_window - json_iter
        global_json_iter += 1
        json_window = get_next_json(global_json_iter)
        json_iter = 0
        
if sql_iter < len_sql_window:
    sql_iter -= 1
    while sql_window:
        right += len_sql_window - sql_iter
        global_sql_iter += STEP_SIZE
        sql_window = get_next_sql(global_sql_iter)
        sql_iter = 0
        
print('Distribution of JOINs:\nonly in JSON: ', left, '\ncommon to SQL and JSON: ', both, '\nonly in SQL: ', right)
fraction_same_title = same_title / both * 1.0
fraction_diff_title = diff_title / both * 1.0

print('''\nFor people with the same EMD5 (total {}):
People with the same job title in SQL and JSON: {}
This is {} of the total number of people with the same EMD5 values

People with the different job title in SQL and JSON: {}
This is {} of the total number of people with the same EMD5 values
'''.format(both, same_title, fraction_same_title, diff_title, fraction_diff_title))

write_dict_to_csv(result)

Distribution of JOINs:
only in JSON:  335372 
common to SQL and JSON:  301031 
only in SQL:  336119

For people with the same EMD5 (total 301031):
People with the same job title in SQL and JSON: 300580
This is 0.9985018154276469 of the total number of people with the same EMD5 values

People with the different job title in SQL and JSON: 451
This is 0.0014981845723530136 of the total number of people with the same EMD5 values



Error: need to escape, but no escapechar set

In [144]:
write_dict_to_csv(result)

# Controller pseudo code

'''
populate window with 10000 rows from SQL table
populate window with 10000 entries from JSON file

initialize iterators as 0 for both SQL and JSON file

iterate over SQL and JSON:
    if emd5 equal:
        center += 1
        check_profession_different()
        increment both iterators
        
    elif json < sql:
        increment json iterator
    elif json > sql:
        increment sql iterator
        
    if json_iterator at end of json_window:
        populate window with 10000 entries from JSON file
        increment global json iterator
    if sql_iterator at end of sql_window:
        populate window with 10000 rows from SQL table
        increment global sql iterator
    
    if either is at end of the 63000:
        break

if json_iterator < len(json_window):
    left += remaining json elements
elif sql_iterator < len(sql_window):
    right += remaining sql elements

perform post processing
'''

In [108]:
json_id_set = set()
for page_number in range(1, 65):
    json_page = get_next_json(page_number)
    for json_obj in json_page:
        if json_obj['id'] in json_id_set:
            print('repeat: ', json_obj['id'])
        json_id_set.add(json_obj['id'])

In [112]:
xyz = {1:1, 2:2}
import json
print(json.dumps(xyz))

{"1": 1, "2": 2}
